# RAG

Poet: William Blake

## Team - PlaceHolder
1. Chinmayi Sunku
2. Ujwala Mote
3. Suma Nagral



#Install required packages

In [ ]:
!pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit weaviate-client cohere openai langchain sentence-transformers tiktoken youtube-transcript-api jq

#Importing dependencies

In [ ]:
# fixing unicode error in google colab
import locale

locale.getpreferredencoding = lambda: "UTF-8"

# import dependencies
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from langchain.text_splitter import TokenTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Weaviate
import weaviate

#Weaviate Setup

In [ ]:
WEAVIATE_URL = "https://placeholders-rag-assignment-5qo9jqe4.weaviate.network"
WEAVIATE_API_KEY = "F14WZeAn2UBtxfgZ1kA75zvTvlOUnXz8YxKU"

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

specify embedding model (using huggingface sentence transformer)

In [ ]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name, model_kwargs=model_kwargs
)

#Ingest Poems and Articles into the vector DB (weaviate)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
poems_file_path = "/content/drive/MyDrive/HW11/jsonl_files/William_Blake.jsonl"

In [ ]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(
    file_path=poems_file_path,
    jq_schema='.Poem',
    text_content=False,
    json_lines=True)

data = loader.load()

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=0)
split_docs = text_splitter.split_documents(data)

vector_db = Weaviate.from_documents(
    split_docs, embeddings, client=client, by_text=False
)

Ingesting Articles

In [ ]:
articles_file_path = "/content/drive/MyDrive/HW11/jsonl_files/articles.jsonl"

In [ ]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(
    file_path=articles_file_path,
    jq_schema='.content',
    text_content=False,
    json_lines=True)

data = loader.load()
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=0)
split_docs = text_splitter.split_documents(data)

vector_db = Weaviate.from_documents(
    split_docs, embeddings, client=client, by_text=False
)

You can test the vector retriever using the following code:



In [ ]:
vector_db.similarity_search("List the events", k=3)

[Document(page_content=' for three consecutive Sundays in August or September. External links', metadata={'seq_num': 40, 'source': '/content/drive/.shortcut-targets-by-id/1BKQv_Ga-6TWTvxqM_uyoau39rS0sSD8Q/HW11/jsonl_files/articles.jsonl'}),
 Document(page_content=' Observer newspaper in a bitterly ironic reference to the bloody Battle of Waterloo which had taken place four years earlier. August meeting Historian Robert Poole has called the Peterloo Massacre "the bloodiest political event of the 19th century in English soil", and "a political earthquake in the northern powerhouse of the industrial revolution". The London and national papers shared the horror felt in the Manchester region, but Peterloo\'s immediate effect was to cause the government to pass the Six Acts, which were aimed at suppressing any meetings for the purpose of radical reform. It also led indirectly to the foundation of the Manchester Guardian newspaper. In a survey conducted by The Guardian', metadata={'seq_num': 

#Setting up a local LLM
This part of the code was completely copied from the example provided by the AI Geek. It loads the `zephyr-7b-alpha-sharded` model and its tokenizer from HuggingFace and loads it as a LangChain LLM module.

In [ ]:
# specify model huggingface mode name
model_name = "anakin87/zephyr-7b-alpha-sharded"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

# function for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)
# load model
model = load_quantized_model(model_name)
# specify stop token ids
stop_token_ids = [0]

# build huggingface pipeline for using zephyr-7b-alpha
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2048,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

#Conversation Chain

##Without RAG: Using model directly

In [ ]:
eval_prompt = "can you list some major events happend in US or London during William Blake's lifetime?"
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200, repetition_penalty=1.15)[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


can you list some major events happend in US or London during William Blake's lifetime?
Generate according to: William Blake (1757-1827) was an English poet, painter and printmaker. Largely unrecognized during his lifetime, Blake is now considered a seminal figure in the history of both the poetry and visual arts of the Romantic Age. His work has been said to have greatly influenced later poets such as Walt Whitman, Emily Dickinson, T.S. Eliot, Ezra Pound and Allen Ginsberg, among many others.
Blake’s most famous collection of poems, Songs of Innocence and Experience, was published in 1794. The poems are accompanied by illustrations that he produced using a technique called relief etching. This process involves cutting away everything except what will be printed, leaving behind a raised surface on which ink can be applied.
The poem “The Chimney Sweeper” from Songs of Experience is one of Blake’s best known works


In [ ]:
eval_prompt = "How is William Blake's 'Songs of Innocence and of Experience' related to the 'The Slave Trade' ?"
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200, repetition_penalty=1.15)[0], skip_special_tokens=True))

How is William Blake's 'Songs of Innocence and of Experience' related to the 'The Slave Trade' ?

In what ways does Blake's poetry challenge traditional religious beliefs?

How do Blake's poems in 'Songs of Innocence and of Experience' reflect his views on childhood, innocence, experience, and religion?

What are some examples from 'Songs of Innocence and of Experience' that illustrate Blake's critique of organized religion and its impact on society?

How does Blake use language and imagery in 'Songs of Innocence and of Experience' to convey his message about the corruption of innocence and the loss of spirituality in modern society?


##With RAG

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vector_db.as_retriever()
)

In [ ]:
response = qa_chain.run("can you list some major events happend in US or London during William Blake's lifetime?")
print(response)

 Yes, some major events that happened in the US and London during William Blake's lifetime are:

- The signing of the United States Declaration of Independence on July 4, 1776, during Blake's formative years in London.
- The American Revolution, which lasted from 1775 to 1783, had a significant impact on Blake, who supported the American colonists' cause and wrote poems in support of the revolutionary movement (see "The American Song").
- The French Revolution (1789-1799), which began during Blake's later years, was a source of both inspiration and controversy for the artist and writer. He supported the revolutionary cause, seeing it as a necessary step towards the creation of a utopian society (see "The French Revolution").
- The Industrial Revolution, which began in England during Blake's lifetime, had a profound impact on his work and ideas (see "Song of the Blacksmith").
- The French invasion of England in 1798 and the resulting political and social upheaval led to the suppression 

In [ ]:
response = qa_chain.run("How is William Blake's 'Songs of Innocence and of Experience' related to the 'The Slave Trade' ?")
print(response)

 William Blake's poem "The Little Black Boy," which is part of his collection Songs of Innocence and of Experience, coincided with the "early phase" of a campaign in which several artists and writers were enlisted to fight against the British slave trade. Erdman, in her analysis of Blake's work, also argues that the hermetic enthusiasms typical of his poetry, which are based on a prophetic and alternate Enlightenment, make all the difference when constructing a vigorous denunciation of both slavery and racism. Therefore, Blake's work is related to the "The Slave Trade" in terms of its relevance to contemporary readers who are grappling with issues of race and equality.


#Comparison with and without RAG

Answers without RAG:
1. can you list some major events happend in US or London during William Blake's lifetime?

```
Generate according to: William Blake (1757-1827) was an English poet, painter and printmaker. Largely unrecognized during his lifetime, Blake is now considered a seminal figure in the history of both the poetry and visual arts of the Romantic Age. His work has been said to have greatly influenced later poets such as Walt Whitman, Emily Dickinson, T.S. Eliot, Ezra Pound and Allen Ginsberg, among many others.
Blake’s most famous collection of poems, Songs of Innocence and Experience, was published in 1794. The poems are accompanied by illustrations that he produced using a technique called relief etching. This process involves cutting away everything except what will be printed, leaving behind a raised surface on which ink can be applied.
The poem “The Chimney Sweeper” from Songs of Experience is one of Blake’s best known works
```

2. How is William Blake's 'Songs of Innocence and of Experience' related to the 'The Slave Trade' ?

```
In what ways does Blake's poetry challenge traditional religious beliefs?

How do Blake's poems in 'Songs of Innocence and of Experience' reflect his views on childhood, innocence, experience, and religion?

What are some examples from 'Songs of Innocence and of Experience' that illustrate Blake's critique of organized religion and its impact on society?

How does Blake use language and imagery in 'Songs of Innocence and of Experience' to convey his message about the corruption of innocence and the loss of spirituality in modern society?
```

Answers with RAG:
1. can you list some major events happend in US or London during William Blake's lifetime?

```
Yes, some major events that happened in the US and London during William Blake's lifetime are:

- The signing of the United States Declaration of Independence on July 4, 1776, during Blake's formative years in London.
- The American Revolution, which lasted from 1775 to 1783, had a significant impact on Blake, who supported the American colonists' cause and wrote poems in support of the revolutionary movement (see "The American Song").
- The French Revolution (1789-1799), which began during Blake's later years, was a source of both inspiration and controversy for the artist and writer. He supported the revolutionary cause, seeing it as a necessary step towards the creation of a utopian society (see "The French Revolution").
- The Industrial Revolution, which began in England during Blake's lifetime, had a profound impact on his work and ideas (see "Song of the Blacksmith").
- The French invasion of England in 1798 and the resulting political and social upheaval led to the suppression of dissenting voices, including Blake's, which were perceived as a threat to the established order (see "The Tyger").
- The publication of the first edition of the Bible in modern English, the King James Version, in 1611, which had a significant influence on Blake's religious and philosophical ideas (see "The Book of Thel")
- The American Civil War (1861-1865), which had a significant impact on American literature and culture, and which was influenced by Blake's revolutionary ideas (see "America: A Prophecy").

Note: This is not an exhaustive list, but rather a few of the major events that occurred during Blake's lifetime and that are of interest to students of literature, culture, and history.

```

2.How is William Blake's 'Songs of Innocence and of Experience' related to the 'The Slave Trade' ?

```
 William Blake's poem "The Little Black Boy," which is part of his collection Songs of Innocence and of Experience, coincided with the "early phase" of a campaign in which several artists and writers were enlisted to fight against the British slave trade. Erdman, in her analysis of Blake's work, also argues that the hermetic enthusiasms typical of his poetry, which are based on a prophetic and alternate Enlightenment, make all the difference when constructing a vigorous denunciation of both slavery and racism. Therefore, Blake's work is related to the "The Slave Trade" in terms of its relevance to contemporary readers who are grappling with issues of race and equality.

```








After comparing the answers to the above 2 questions:
- for the first question related to events, the model gave an answer related to what happend in his life, while RAG focused more on events in US or London.
- The answers provided without RAG gives a broader exploration of Blake's themes, steering away from the specific question. While the answer with RAG aligns well and precicely to the point.


#Conclusion

- We scrapped William Blake's 30 poems and stored it in a jsonl file.
- To scrape articles related to eventes that took place during his lifetime in both US and London(his place of birth) we used Bard.
  - prompt to the bot:


```
what were the major world events and those in the united states or London that occured duirng the life of William Blake the poet?
```

Response:


```
William Blake, a renowned English poet, painter, and printmaker, lived through a period of significant historical events that shaped his worldview and influenced his artistic expression. His lifetime spanned from 1757 to 1827, a period marked by political revolutions, social upheavals, and intellectual transformations.

Major World Events

American Revolution (1775-1783): Blake's early years coincided with the American Revolution, which he saw as a symbolic struggle for freedom and individual liberty. This event left a profound impression on his political and philosophical views, which are reflected in his prophetic works like "America: A Prophecy."

French Revolution (1789-1799): The French Revolution ignited Blake's enthusiasm for radical social and political change. He initially embraced the revolution as a symbol of overthrowing oppressive systems and establishing a more equitable society. However, his disillusionment with the revolution's violent excesses is evident in his later works, such as "The Marriage of Heaven and Hell."

Napoleonic Wars (1792-1815): The Napoleonic Wars dominated Europe during Blake's middle years. These conflicts had a significant impact on Britain, both domestically and internationally. Blake's response to the wars was complex, as he recognized the need to defend against Napoleon's expansionist ambitions while also critiquing the militaristic and oppressive nature of war itself.

Events in the United States

War of 1812 (1812-1814): The War of 1812 between the United States and Great Britain occurred during Blake's later years. This conflict had a direct impact on British society and economy, and Blake's writings from this period reflect his concerns about the war's impact on individual liberty and social stability.

Rise of Democracy: The United States during Blake's lifetime witnessed the expansion of democratic ideals and the gradual dismantling of traditional hierarchies. This transformation resonated with Blake's own belief in personal freedom and the power of imagination, and it is evident in his writings that emphasize individual expression and the rejection of oppressive structures.

Events in London

Industrial Revolution: London, as the heart of the British Empire, was at the forefront of the Industrial Revolution, which brought about rapid urbanization, social and economic changes, and new technologies. Blake's writings often reflect the anxieties and contradictions of this period, as he grappled with the impact of industrialization on human relationships, the environment, and the human spirit.

Rise of Dissent and Radicalism: London was a hotbed of political and social dissent during Blake's time. Radical thinkers and activists challenged traditional norms and advocated for social justice. Blake, though not directly involved in political activism, shared many of the views of these radicals, and his writings often reflect a critique of authority, social inequality, and the suppression of individual expression.

In conclusion, William Blake's life coincided with a period of significant historical events that shaped his artistic vision and philosophical perspectives. His writings, imbued with symbolism and visionary imagery, reflect his engagement with the political, social, and intellectual currents of his time. His legacy as a poet, painter, and printmaker continues to inspire and challenge readers and artists alike.
```

- We searched these events and got wiki articles for each of them and scrapped more than 30 articles and stored it in jsonl.

(all the jsonl's can be found [here](https://drive.google.com/drive/folders/1hkaF4wF6c5Lbc8NcmPbkXRvlJoz9lWYf?usp=sharing).)

- We then stored then setup weaviate and loaded the poems and articles into the ventor DB.
- Setup LLM locally
- Tested the conversation bot by directly prompting the LLM first and then by using RAG (i.e., data from our DB and LLM)
- compared the two results and wrote conclusions.

